**1단계: 코랩 환경 설정 및 라이브러리 설치**

코랩에서 코드를 실행하려면 필요한 라이브러리들을 먼저 설치해야 합니다. 특히 웹 브라우저를 제어하는 Selenium과 얼굴 검출 라이브러리 mtcnn은 기본으로 설치되어 있지 않습니다.

In [1]:
# 셀레니움, 웹드라이버 관리자, 얼굴 검출 라이브러리를 설치합니다.
!pip install selenium webdriver-manager mtcnn

# 웹 드라이버 실행에 필요한 추가 패키지를 설치합니다.
!apt-get update
!apt-get install -y chromium-chromedriver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 24.9 MB/s eta 0:00:00
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 

**2단계: 라이브러리 불러오기 및 기본 설정**

본격적인 코드 실행에 앞서, 필요한 모든 파이썬 라이브러리를 불러오고, 어떤 이미지를 얼마나 수집할지 기본 환경을 설정합니다.

In [2]:
import os
import time
import json
import requests
import hashlib
import numpy as np
from io import BytesIO
from PIL import Image
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from mtcnn import MTCNN

# -----------------
# 환경 설정
# -----------------
# 수집할 이미지 카테고리 (검색어)
categories = [
    "Korean K-pop idol profile picture",
    "African actor profile picture",
    "European actress profile picture",
    "Hispanic actress profile picture",
    "American model profile picture"
]
# 카테고리별 최대 저장 이미지 수
MAX_IMAGES_PER_CATEGORY = 15
# 저장할 최종 이미지 크기 (가로, 세로)
TARGET_FACE_SIZE = (224, 224)
# 최종 이미지를 저장할 기본 폴더 이름
BASE_OUTPUT_DIR = "dataset_bing_highres_frontal"

**3단계: 얼굴 검출 모델 및 보조 함수 준비**

이미지에서 얼굴을 찾고, 정면을 바라보는 얼굴만 골라내며, 중복 이미지를 걸러내는 등 크롤링에 필요한 여러 가지 보조 기능들을 함수로 미리 만들어 둡니다.

In [3]:
# 1) MTCNN 얼굴 검출 모델을 준비합니다.
detector = MTCNN()

# 2) 얼굴이 정면을 보는지 엄격하게 판단하는 함수
def is_frontal_strict(face):
    keypoints = face["keypoints"]
    eye_diff = abs(keypoints["left_eye"][1] - keypoints["right_eye"][1]) # 양 눈의 높이 차이
    nose_center = (keypoints["left_eye"][0] + keypoints["right_eye"][0]) / 2 # 양 눈의 중심 x좌표
    center_offset = abs(keypoints["nose"][0] - nose_center) # 코와 눈 중심의 x좌표 차이
    return (eye_diff < 15) and (center_offset < 20) # 두 조건이 모두 맞아야 정면으로 인정

# 3) 코랩 환경에 맞는 웹 드라이버를 설정하고 반환하는 함수
def get_colab_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--headless') # 브라우저 창을 띄우지 않음
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    return webdriver.Chrome(options=options)

# 4) 이미지 데이터의 고유한 해시(hash) 값을 만들어 중복을 확인하는 함수
def hash_image(image_data):
    return hashlib.md5(image_data).hexdigest()

**4단계: 이미지 검색, 다운로드 및 얼굴 추출**

가장 중요한 부분으로, 설정된 검색어로 Bing 이미지 검색을 수행하고, 고화질 원본 이미지를 다운로드하여 얼굴을 찾은 뒤, 정면 얼굴만 골라 지정된 크기로 잘라 저장하는 전체 과정을 담은 함수입니다.

In [4]:
def download_images_bing(query, download_path, max_images=10):
    driver = get_colab_driver()
    print(f"[INFO] '{query}' 검색을 시작합니다.")

    # Bing 이미지 검색 URL로 이동
    search_url = f"https://www.bing.com/images/search?q={query.replace(' ', '+')}&form=HDRSC2"
    driver.get(search_url)
    time.sleep(2)

    os.makedirs(download_path, exist_ok=True)
    seen_hashes = set()
    saved_count = 0

    # 썸네일(미리보기 이미지) 목록 가져오기
    thumbnails = driver.find_elements(By.CSS_SELECTOR, "a.iusc")

    for thumb in thumbnails:
        if saved_count >= max_images:
            break

        try:
            # ① 썸네일 정보에서 원본 이미지 주소(URL) 추출
            m_json = thumb.get_attribute("m")
            info = json.loads(m_json)
            img_url = info.get("murl")

            # ② 원본 이미지 다운로드
            resp = requests.get(img_url, timeout=10)
            image_data = resp.content
            pil_orig = Image.open(BytesIO(image_data)).convert("RGB")

            # ③ 다운로드한 이미지에서 얼굴 검출
            img_np = np.array(pil_orig)
            faces = detector.detect_faces(img_np)
            if not faces:
                continue

            # ④ 가장 큰 얼굴 하나만 선택
            main_face = max(faces, key=lambda f: f['box'][2] * f['box'][3])

            # ⑤ 정면 얼굴인지, 중복된 이미지인지 확인
            if not is_frontal_strict(main_face):
                continue

            x, y, w, h = main_face['box']
            face_crop_pil = pil_orig.crop((x, y, x + w, y + h))
            img_hash = hash_image(face_crop_pil.tobytes())

            if img_hash in seen_hashes:
                continue
            seen_hashes.add(img_hash)

            # ⑥ 최종 이미지 리사이즈 및 저장
            final_image = face_crop_pil.resize(TARGET_FACE_SIZE, Image.LANCZOS)
            filename = f"{query.replace(' ', '_')}_{saved_count + 1}.jpg"
            final_image.save(os.path.join(download_path, filename), format="JPEG", quality=95)

            print(f"[SAVE] '{filename}' 저장 완료")
            saved_count += 1

        except Exception as e:
            # 과정 중 오류가 발생하면 건너뛰기
            continue

    driver.quit()
    print(f"[DONE] '{query}' 완료. 총 {saved_count}개 이미지 저장.\n")

**5단계: 전체 프로세스 실행**

이제 위에서 만든 모든 기능을 조합하여, categories 리스트에 정의된 각 주제에 대해 이미지 수집을 시작합니다.

In [ ]:
if __name__ == "__main__":
    for cat in categories:
        # 카테고리별로 저장 폴더 생성
        folder_name = cat.replace(" ", "_")
        out_dir = os.path.join(BASE_OUTPUT_DIR, folder_name)

        # 해당 카테고리에 대한 이미지 다운로드 함수 호출
        download_images_bing(cat, out_dir, max_images=MAX_IMAGES_PER_CATEGORY)

[INFO] 'Korean K-pop idol profile picture' 검색을 시작합니다.
[SAVE] 'Korean_K-pop_idol_profile_picture_1.jpg' 저장 완료
[SAVE] 'Korean_K-pop_idol_profile_picture_2.jpg' 저장 완료
[SAVE] 'Korean_K-pop_idol_profile_picture_3.jpg' 저장 완료
[SAVE] 'Korean_K-pop_idol_profile_picture_4.jpg' 저장 완료
[SAVE] 'Korean_K-pop_idol_profile_picture_5.jpg' 저장 완료
